<a href="https://colab.research.google.com/github/HeitorAl/machine-learning-practitioner-dio/blob/main/Transfer_Learning_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline

import shutil
import os
import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from sklearn.model_selection import train_test_split
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout

### Atenção

A caixa abaixo foi executada apenas para criar os arquivos de teste e treino no drive, não é necessário executá-la novamente

In [ ]:
'''
# Caminhos de entrada e saída
base_dir = "drive/My Drive/Colab Notebooks/PetImages"  # Caminho da pasta com as subpastas 'cats' e 'dogs'
output_dir = "drive/My Drive/Colab Notebooks/PetImages"  # Caminho para salvar os conjuntos de treino e teste

# Listar imagens e rótulos
image_paths = []
labels = []

for label in os.listdir(base_dir):  # Classes, como 'cats' e 'dogs'
    class_dir = os.path.join(base_dir, label)
    for img in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, img))
        labels.append(label)

# Divisão usando train_test_split
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, train_size=0.7, stratify=labels, random_state=42
)

# Copiar arquivos para os diretórios de treino e teste
for path, label in zip(train_paths, train_labels):
    train_dir = os.path.join(output_dir, "train", label)
    os.makedirs(train_dir, exist_ok=True)
    shutil.copy(path, os.path.join(train_dir, os.path.basename(path)))

for path, label in zip(test_paths, test_labels):
    test_dir = os.path.join(output_dir, "test", label)
    os.makedirs(test_dir, exist_ok=True)
    shutil.copy(path, os.path.join(test_dir, os.path.basename(path)))

print("Conjuntos de treino e teste criados com sucesso!")
'''

Conjuntos de treino e teste criados com sucesso!


In [3]:
model = MobileNet(weights="imagenet", include_top=False)

<ipython-input-3-95e9054bab38>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNet(weights="imagenet", include_top=False)


In [4]:
x = model.output
x = GlobalAveragePooling2D()(x)

In [5]:
x = Dense(50, activation='relu')(x)
preds = Dense(1, activation='sigmoid')(x)
model = Model(inputs=model.input, outputs=preds)

In [6]:
for i, layer in enumerate(model.layers):
  print(i, layer.name)

0 input_layer
1 conv1
2 conv1_bn
3 conv1_relu
4 conv_dw_1
5 conv_dw_1_bn
6 conv_dw_1_relu
7 conv_pw_1
8 conv_pw_1_bn
9 conv_pw_1_relu
10 conv_pad_2
11 conv_dw_2
12 conv_dw_2_bn
13 conv_dw_2_relu
14 conv_pw_2
15 conv_pw_2_bn
16 conv_pw_2_relu
17 conv_dw_3
18 conv_dw_3_bn
19 conv_dw_3_relu
20 conv_pw_3
21 conv_pw_3_bn
22 conv_pw_3_relu
23 conv_pad_4
24 conv_dw_4
25 conv_dw_4_bn
26 conv_dw_4_relu
27 conv_pw_4
28 conv_pw_4_bn
29 conv_pw_4_relu
30 conv_dw_5
31 conv_dw_5_bn
32 conv_dw_5_relu
33 conv_pw_5
34 conv_pw_5_bn
35 conv_pw_5_relu
36 conv_pad_6
37 conv_dw_6
38 conv_dw_6_bn
39 conv_dw_6_relu
40 conv_pw_6
41 conv_pw_6_bn
42 conv_pw_6_relu
43 conv_dw_7
44 conv_dw_7_bn
45 conv_dw_7_relu
46 conv_pw_7
47 conv_pw_7_bn
48 conv_pw_7_relu
49 conv_dw_8
50 conv_dw_8_bn
51 conv_dw_8_relu
52 conv_pw_8
53 conv_pw_8_bn
54 conv_pw_8_relu
55 conv_dw_9
56 conv_dw_9_bn
57 conv_dw_9_relu
58 conv_pw_9
59 conv_pw_9_bn
60 conv_pw_9_relu
61 conv_dw_10
62 conv_dw_10_bn
63 conv_dw_10_relu
64 conv_pw_10
65 conv_

In [7]:
for layer in model.layers[:88]:
  layer.trainable=False
for layer in model.layers[88:]:
  layer.trainable=True

In [8]:
batch_size = 32

In [9]:
from PIL import Image

def verify_images(data_dir):
  """
  Verifies the integrity of images in a given directory.

  Args:
    data_dir: The path to the directory containing image data.

  Raises:
    ValueError: If an image cannot be opened by PIL.
  """
  for root, _, files in os.walk(data_dir):
    for file in files:
      if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        try:
          filepath = os.path.join(root, file)
          img = Image.open(filepath)  # Attempt to open with PIL
          img.verify()  # Verify image integrity
          img.close()
        except (IOError, SyntaxError) as e:
          print(f"Error with image: {filepath} - {e}")
          # Consider removing or replacing the problematic image
          os.remove(filepath)  # Uncomment to remove corrupted image

# Verify images in your training and test directories
verify_images('drive/MyDrive/Colab Notebooks/PetImages/train')
verify_images('drive/MyDrive/Colab Notebooks/PetImages/test')

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [10]:

# Função personalizada para ignorar erros ao carregar batches
def safe_flow_from_directory(datagen, directory, target_size, batch_size, class_mode):
    generator = datagen.flow_from_directory(
        directory=directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode
    )
    while True:  # Loop para manter o gerador ativo
        try:
            batch = next(generator)
            yield batch
        except Exception as e:
            print(f"Erro ao carregar lote: {e}")
            continue

# Configurar o ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Criar os geradores seguros para treinamento e teste
training_set = safe_flow_from_directory(
    datagen=train_datagen,
    directory='drive/MyDrive/Colab Notebooks/PetImages/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

test_set = safe_flow_from_directory(
    datagen=test_datagen,
    directory='drive/MyDrive/Colab Notebooks/PetImages/test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
"""
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.4,
                                   zoom_range = 0.4,
                                   height_shift_range = 0.3,
                                   width_shift_range = 0.3,
                                   rotation_range = 50,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('drive/MyDrive/Colab Notebooks/PetImages/train',
                                                 target_size = (224,224),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('drive/MyDrive/Colab Notebooks/PetImages/test',
                                            target_size = (224,224),
                                            batch_size = batch_size,
                                            class_mode = 'binary')
"""

"\ntrain_datagen = ImageDataGenerator(rescale = 1./255,\n                                   shear_range = 0.4,\n                                   zoom_range = 0.4,\n                                   height_shift_range = 0.3,\n                                   width_shift_range = 0.3,\n                                   rotation_range = 50,\n                                   horizontal_flip = True)\n\ntest_datagen = ImageDataGenerator(rescale = 1./255)\n\ntraining_set = train_datagen.flow_from_directory('drive/MyDrive/Colab Notebooks/PetImages/train',\n                                                 target_size = (224,224),\n                                                 batch_size = batch_size,\n                                                 class_mode = 'binary')\n\ntest_set = test_datagen.flow_from_directory('drive/MyDrive/Colab Notebooks/PetImages/test',\n                                            target_size = (224,224),\n                                            batch_

In [11]:
model.compile(optimizer=Adam(learning_rate = 0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(x=training_set,
                   steps_per_epoch = int(8000/batch_size),
                   epochs=10,
                   validation_data = test_set,
                   validation_steps = int(2000/batch_size))

Found 17441 images belonging to 2 classes.
Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 889ms/step - accuracy: 0.4854 - loss: 1.1819Found 7474 images belonging to 2 classes.
250/250 ━━━━━━━━━━━━━━━━━━━━ 298s 1s/step - accuracy: 0.4854 - loss: 1.1817 - val_accuracy: 0.4798 - val_loss: 1.0795
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 89s 358ms/step - accuracy: 0.4616 - loss: 1.0625 - val_accuracy: 0.4748 - val_loss: 0.9775
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 81s 326ms/step - accuracy: 0.4865 - loss: 0.9988 - val_accuracy: 0.4778 - val_loss: 0.9861
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 83s 331ms/step - accuracy: 0.4902 - loss: 0.9544 - val_accuracy: 0.4873 - val_loss: 0.9237
Epoch 5/10
189/250 ━━━━━━━━━━━━━━━━━━━━ 15s 253ms/step - accuracy: 0.5042 - loss: 0.9083

In [ ]:
model.save('transfer_learning_mobilenet.keras')
from google.colab import files
files.download('transfer_learning_mobilenet.keras')

In [ ]:
from skimage import io

url = "https://super.abril.com.br/wp-content/uploads/2024/11/1811-cachorro-layout_site.jpg?quality=70&strip=info&w=1024&h=682&crop=1"
image = io.imread(url)

plt.imshow(image)
plt.title("Imagem obtida")
plt.tight_layout()


In [ ]:
import cv2

test_image = image
teste_image = cv2.resize(image, (224,224))
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image/255

if model.predict(test_image)[0][0] > 0.5:
  print("Previsão: cachorro")
else:
  print("Previsão: gato")